# Achievement 4, Part 1

## Table of Contents

> 1. Import Libraries
> 2. Import Data
> 3. Check for PII
> 4. Create Regions
> 5. Exclusion Flag for Low-Activity Customers
> 6. Export Subset
> 7. Create Profiling Columns
> 8. Export DF

### 01.Import Libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

### 02.Import Data

In [2]:
#create path
path = r'/Users/juliewhitton/Desktop/Instacart Basket Analysis/'

In [3]:
#import final_data_set
final_data_set = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'final_data_set.pkl'))



In [4]:
#check import
final_data_set.head()

,order_id,user_id,eval_set,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,...,freq_flag,first_name,last_name,gender,state,age,date_joined,number_dependents,family_status,income
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [5]:
#remove eval_set

df = final_data_set.drop(columns = ['eval_set'])

In [6]:
df.dtypes


order_id                          int64
user_id                          object
order_number                      int64
order_day_of_week                 int64
time_of_order                     int64
days_since_prior_order          float64
product_id                        int64
add_to_cart_order                 int64
reordered                         int64
product_name                     object
aisle_id                          int64
department_id                     int64
prices                          float64
price_range_loc                  object
busiest_day                      object
busiest_days                     object
busiest_period_of_day            object
max_order                         int64
loyalty_flag                     object
avg_spent                       float64
spending_flag                    object
median_days_since_last_order    float64
freq_flag                        object
first_name                       object
last_name                        object


In [7]:
df[['order_number', 'order_day_of_week', 'time_of_order',
   'reordered', 'aisle_id', 'max_order', 'age', 'number_dependents', 
   'department_id', 'add_to_cart_order']] = df[['order_number', 'order_day_of_week', 'time_of_order',
                                                'reordered', 'aisle_id', 'max_order', 'age', 'number_dependents', 
                                                'department_id', 'add_to_cart_order']].astype('int8')

In [8]:
df[['order_id', 'product_id', 'income', 'user_id']] = df[['order_id', 'product_id', 'income',
                                                         'user_id']].astype('int32')

In [9]:
df[['median_days_since_last_order', 'avg_spent', 'prices', 
    'days_since_prior_order']] = df[['median_days_since_last_order', 'avg_spent', 
                                     'prices', 'days_since_prior_order']].astype('float16')

### 03.Check for PII

In [10]:
#check head of df
df.head()

,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,freq_flag,first_name,last_name,gender,state,age,date_joined,number_dependents,family_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [11]:
#check shape of df
df.shape

(32404859, 32)

### 04.Create Regions to analyze data

In [12]:
#check value counts of 'state'
df['state'].value_counts(dropna = False)

Pennsylvania            667082
California              659783
Rhode Island            656913
Georgia                 656389
New Mexico              654494
Arizona                 653964
North Carolina          651900
Oklahoma                651739
Alaska                  648495
Minnesota               647825
Massachusetts           646358
Wyoming                 644255
Virginia                641421
Missouri                640732
Texas                   640394
Colorado                639280
Maine                   638583
North Dakota            638491
Alabama                 638003
Kansas                  637538
Louisiana               637482
Delaware                637024
South Carolina          636754
Oregon                  636425
Arkansas                636144
Nevada                  636139
New York                635983
Montana                 635265
South Dakota            633772
Illinois                633024
Hawaii                  632901
Washington              632852
Mississi

In [13]:
#Create a regional segmentation of the data.

northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 
             'New York', 'Pennsylvania', 'New Jersey']
midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 
           'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
south = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 
         'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Michigan', 
         'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana', 'Mississippi']
west = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 
        'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [14]:
#You’ll need to create a “Region” column based on the “State” column from your customers data set.
 

df.loc[df['state'].isin(northeast), 'region'] = 'Northeast'
df.loc[df['state'].isin(midwest), 'region'] = 'Midwest'
df.loc[df['state'].isin(south), 'region'] = 'South'
df.loc[df['state'].isin(west), 'region'] = 'West'

In [15]:
df['region'].value_counts(dropna = False)

South        11422813
West          8292913
Midwest       6966397
Northeast     5722736
Name: region, dtype: int64

In [16]:
#check new column
df.head()

,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,last_name,gender,state,age,date_joined,number_dependents,family_status,income,region
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South


Determine whether there’s a difference in spending habits between the different U.S. regions.

In [17]:
#create a crosstab between region and spending flag
crosstab_spending = pd.crosstab(df['region'], df['spending_flag'], dropna = False)
crosstab_spending

spending_flag,High spender,Low spender
region,,
Midwest,141516,6824881
Northeast,108225,5614511
South,224150,11198663
West,160354,8132559


In [18]:
crosstab_spending.to_clipboard()

### 05.Create an exclusion flag for low-activity customers

In [19]:
#Create an exclusion flag for low-activity customers (customers with less than 5 orders) 
#and exclude them from the data. 

df.loc[df['max_order'] < 5, 'activity_flag'] = "Low-activity Customer"
df.loc[df['max_order'] >=5, 'activity_flag'] = "High-activity Customer"

In [20]:
#check activity_flag column
df['activity_flag'].value_counts(dropna = False)

High-activity Customer    30964564
Low-activity Customer      1440295
Name: activity_flag, dtype: int64

### 06.Exclude low-activity customers and export the subset

In [21]:
#create df excluding low-activity customers
df_high_activity = df.loc[df['activity_flag'] == 'High-activity Customer']

In [22]:
#check new df
df_high_activity.head()

,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,last_name,gender,state,age,date_joined,number_dependents,family_status,income,region,activity_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity Customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity Customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity Customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity Customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South,High-activity Customer


In [23]:
df_low_activity = df.loc[df['activity_flag'] == 'Low-activity Customer']

In [24]:
df_high_activity['activity_flag'].value_counts(dropna = False)

High-activity Customer    30964564
Name: activity_flag, dtype: int64

In [25]:
#export high-activity subset
df_high_activity.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'high_activity.pkl'))

In [26]:
#export low_activity subset
df_low_activity.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'low_activity.pkl'))

In [27]:
df = df_high_activity

### 07.Create a Profiling Variable 

Create a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. You might also use the “orders_day_of_the_week” and “order_hour_of_day” columns if you can think of a way they would impact customer profiles.

In [28]:
#create age groups
df['age'].describe()

count    3.096456e+07
mean     4.946803e+01
std      1.848528e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: age, dtype: float64

Ages range between 18 and 81. I will create 3 classifications: Young Adult (18-34), Adult (35-64), Senior (65-81)

In [29]:
#create new column and categories
df.loc[(df['age'] >= 18) & (df['age'] < 35), 'age_category'] = 'Young Adult'
df.loc[(df['age'] >= 35) & (df['age'] <65), 'age_category'] = 'Adult'
df.loc[(df['age'] >= 65), 'age_category'] = 'Senior'

df['age_category'].value_counts(dropna = False)

Adult          14526725
Young Adult     8242295
Senior          8195544
Name: age_category, dtype: int64

The yearly income of customers ranges between 25,903 and 593,901. I will create 3 income categories based on data from the Pew Research Center: low, middle, and high income.

In [30]:
#explore income column
df['income'].describe()

count    3.096456e+07
mean     9.967587e+04
std      4.314187e+04
min      2.590300e+04
25%      6.729200e+04
50%      9.676500e+04
75%      1.281020e+05
max      5.939010e+05
Name: income, dtype: float64

In [31]:
df['income'].min()

25903

In [32]:
df['income'].max()

593901

In [33]:
#create new column for income category

df.loc[(df['income'] < 52200), 'income_category'] = 'Low income'
df.loc[(df['income'] >= 52200) & (df['income'] < 156600), 'income_category'] = 'Middle income'
df.loc[(df['income'] >= 156600), 'income_category'] = 'High income'

df['income_category'].value_counts(dropna = False)

Middle income    24541333
Low income        3691708
High income       2731523
Name: income_category, dtype: int64

In [34]:
#shape check
df.shape

(30964564, 36)

Create dictionary for department_id so that I can continue creating the profile.

In [35]:
#import department id data
df_depts = pd.read_csv(os.path.join(path, '02 Data','Prepared Data','departments_wrangled.csv'), 
                       index_col = False)

In [36]:
#check import
df_depts

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [37]:
#rename 'Unnamed' col to dept_id
df_depts.rename(columns = {'Unnamed: 0' : 'dept_id'}, inplace = True)

In [38]:
#create dictionary

data_dict = dict(zip(df_depts['dept_id'], df_depts['department']))

In [39]:
#check dictionary
data_dict

{1: 'frozen',
 2: 'other',
 3: 'bakery',
 4: 'produce',
 5: 'alcohol',
 6: 'international',
 7: 'beverages',
 8: 'pets',
 9: 'dry goods pasta',
 10: 'bulk',
 11: 'personal care',
 12: 'meat seafood',
 13: 'pantry',
 14: 'breakfast',
 15: 'canned goods',
 16: 'dairy eggs',
 17: 'household',
 18: 'babies',
 19: 'snacks',
 20: 'deli',
 21: 'missing'}

In [40]:
#rename department_id column and assign department from data dictionary
df['department'] = df['department_id'].map(data_dict)

In [41]:
#check output
df.head()

,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,age,date_joined,number_dependents,family_status,income,region,activity_flag,age_category,income_category,department
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,31,2/17/2019,3,married,40423,South,High-activity Customer,Young Adult,Low income,beverages
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,31,2/17/2019,3,married,40423,South,High-activity Customer,Young Adult,Low income,beverages
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,31,2/17/2019,3,married,40423,South,High-activity Customer,Young Adult,Low income,beverages
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,31,2/17/2019,3,married,40423,South,High-activity Customer,Young Adult,Low income,beverages
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,31,2/17/2019,3,married,40423,South,High-activity Customer,Young Adult,Low income,beverages


In [42]:
#check output
df['department'].value_counts(dropna = False)

produce            9079273
dairy eggs         5177182
snacks             2766406
beverages          2571901
frozen             2121731
pantry             1782705
bakery             1120828
canned goods       1012074
deli               1003834
dry goods pasta     822136
household           699857
meat seafood        674781
breakfast           670850
personal care       424306
babies              410392
international       255991
alcohol             144627
pets                 93060
missing              64768
other                34411
bulk                 33451
Name: department, dtype: int64

In [43]:
#create unique user df
df_cust = df.drop_duplicates(subset = ['user_id'])

df_cust.head()

,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,age,date_joined,number_dependents,family_status,income,region,activity_flag,age_category,income_category,department
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,31,2/17/2019,3,married,40423,South,High-activity Customer,Young Adult,Low income,beverages
59,2968173,15,15,1,9,7.0,196,2,0,Soda,...,69,6/3/2019,0,divorced/widowed,54313,Midwest,High-activity Customer,Senior,Middle income,beverages
131,2293453,19,2,5,14,6.0,196,3,0,Soda,...,62,2/3/2017,1,married,151988,South,High-activity Customer,Adult,Middle income,beverages
335,1573906,21,10,3,10,6.0,196,2,0,Soda,...,71,6/18/2019,0,divorced/widowed,124643,West,High-activity Customer,Senior,Middle income,beverages
540,1593000,31,10,3,8,7.0,196,17,0,Soda,...,38,1/22/2019,1,married,52311,South,High-activity Customer,Adult,Middle income,beverages


I will create a column determining those customers that do and don't have pets and/or babies and one according to number of dependents.

In [44]:
#create new col for pet owner flag

df.loc[df['department'] == 'pets', 'pet_owner'] = 'Yes'
df.loc[df['department'] != 'pets', 'pet_owner'] = 'No'

df['pet_owner'].value_counts(dropna = False)

No     30871504
Yes       93060
Name: pet_owner, dtype: int64

In [45]:
#create new column for customers with babies
df.loc[(df['department'] == 'babies') & (df['number_dependents'] >= 1), 'has_babies'] = 'Yes'
df.loc[df['department'] != 'babies', 'has_babies'] = 'No'
df['has_babies'] = df['has_babies'].fillna('No')

df['has_babies'].value_counts(dropna = False)

No     30657500
Yes      307064
Name: has_babies, dtype: int64

In [46]:
#create column to indicate if customer has dependents or not
df.loc[df['number_dependents'] == 0, 'has_kids'] = 'No'
df.loc[df['number_dependents'] != 0, 'has_kids'] = 'Yes'

df['has_kids'].value_counts(dropna = False)

Yes    23224883
No      7739681
Name: has_kids, dtype: int64

### 08. Export DF

In [47]:
#export df
df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'full_set_with_flags.pkl'))

In [48]:
#export unique customer df
df_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'cust_unique.pkl'))